In [73]:
import torch # ML framework: It use tensor DS for math and calculus
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# Hyperpara
block_size = 8
batch_size = 4 # How many blocks we process in ||

cuda


In [74]:
with open('data/magic_of_oz.txt', 'r', encoding='utf-8') as file:
    text = file.read()
chars = sorted(set(text))
vocab_size = len(chars)
print(chars)
print(vocab_size)



['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
80


In [75]:
# tokenization -> encoding/deconding encoding convert each char to int

string_to_int = { char:index for index,char in enumerate(chars) }
int_to_string = { index:char for index,char in enumerate(chars) }
encode = lambda string: [string_to_int[char] for char in string]
decode = lambda int_list: ''.join([int_to_string[int] for int in int_list])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])



tensor([79,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 50, 33, 64, 64, 73, 71,
        72, 70, 53, 72, 61, 67, 66, 51,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1, 44, 60, 57,  1, 30, 53, 65, 67, 73, 71,  1, 39, 78,  1, 26,
        67, 67, 63, 71,  0,  0,  0, 43, 61, 66])


In [76]:
# Validation and training split
n = int(0.08 * len(data))
train_data = data[:n]
validation_data = data[n:]



In [77]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} target is {target}")

When input is tensor([79]) target is 1
When input is tensor([79,  1]) target is 1
When input is tensor([79,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1,  1,  1,  1]) target is 1
When input is tensor([79,  1,  1,  1,  1,  1,  1,  1]) target is 1
